The below UDF can currently only be used for the CLASSY_WALK_2024 page in EN. This is because EN requires you to first upload a template for your data structure with a column named for the specific page you're processing supporters to. All columns must be mapped appropriately.

Moving forward, we will need to create a new column for each EN page and create an updated format prior to using this UDF with any additional page.

Note: The EN "format name" is hardcoded. The EN page also has an "Override page name" set, which allows us to use CLASSY_WALK_2024 instead of "Classy Walk 2024" as the column name.

To do: The UDF works for a static file name and for processing a single file. Next we need to process anything in the view Jonathan is creating that is not in the processed table.

In [ ]:
CREATE OR REPLACE FUNCTION PROCESS_PAGE()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
HANDLER = 'process_page'
EXTERNAL_ACCESS_INTEGRATIONS = (EN_INTEGRATION)
PACKAGES = ('requests')
SECRETS = ('cred' = EN_API_DB_20241007)
IMPORTS=('@NEW_FILES/20241126_2_P2P_EN_page_interactions.csv')
AS
$$
import requests
import _snowflake
import sys
import os

file_name = "20241126_2_P2P_EN_page_interactions.csv"

# File to upload
with open(os.path.join(sys._xoptions["snowflake_import_directory"], file_name), "r", encoding='utf-8') as f:
    file = f.read()

def process_page():
        
    # URL for the authentication endpoint
    url = "https://us.engagingnetworks.app/ea-dataservice/import.service"
        
    # Get the API token from Snowflake Secrets
    api_token = _snowflake.get_generic_secret_string('cred')  
    
    # Create a payload for the request
    data = {
        'token': api_token,
        'name': 'Test import from Snowflake',
        'formatName': 'classy_to_EN_API'
    }
    
    # Send the request
    response = requests.post(url, data=data, files={'file': (file_name, file)})

    # Print response
    return response.text  # Return the response content as a string

$$;

In [ ]:
SELECT PROCESS_PAGE()